# Analyze Results of a Hyperparameter Tuning job

Once you have completed a tuning job, (or even while the job is still running) you can use this notebook to analyze the results to understand how each hyperparameter effects the quality of the model.

---
## Set up the environment
To start the analysis, you must pick the name of the hyperparameter tuning job.

In [13]:
import boto3
import sagemaker
import os

region = boto3.Session().region_name
sage_client = boto3.Session().client("sagemaker")


In [15]:
response = sage_client.list_hyper_parameter_tuning_jobs()
tuning_job_name = response['HyperParameterTuningJobSummaries'][0]['HyperParameterTuningJobName']
tuning_job_name

'xgboost-211111-0708'

## Track hyperparameter tuning job progress
After you launch a tuning job, you can see its progress by calling describe_tuning_job API. The output from describe-tuning-job is a JSON object that contains information about the current state of the tuning job. You can call list_training_jobs_for_tuning_job to see a detailed list of the training jobs that the tuning job launched.

In [16]:
# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)

status = tuning_job_result["HyperParameterTuningJobStatus"]
if status != "Completed":
    print("Reminder: the tuning job has not been completed.")

job_count = tuning_job_result["TrainingJobStatusCounters"]["Completed"]
print("%d training jobs have completed" % job_count)

objective = tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]
is_minimize = objective["Type"] != "Maximize"
objective_name = objective["MetricName"]

9 training jobs have completed


In [4]:
from pprint import pprint

if tuning_job_result.get("BestTrainingJob", None):
    print("Best model found so far:")
    pprint(tuning_job_result["BestTrainingJob"])
else:
    print("No training jobs have reported results yet.")

Best model found so far:
{'CreationTime': datetime.datetime(2021, 11, 11, 7, 12, 47, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'validation:auc',
                                                 'Value': 0.7740089893341064},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2021, 11, 11, 7, 16, 47, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:875692608981:training-job/xgboost-211111-0708-004-fed4f418',
 'TrainingJobName': 'xgboost-211111-0708-004-fed4f418',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2021, 11, 11, 7, 15, 47, tzinfo=tzlocal()),
 'TunedHyperParameters': {'alpha': '0.4705200372930718',
                          'eta': '0.26921699408605',
                          'max_depth': '10',
                          'min_child_weight': '1.1647217918469628'}}


## Fetch all results as DataFrame
We can list hyperparameters and objective metrics of all training jobs and pick up the training job with the best objective metric.

In [5]:
import pandas as pd

tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
        pd.set_option("display.max_colwidth", None)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df

Number of training jobs with valid objective: 9
{'lowest': 0.6875259876251221, 'highest': 0.7740089893341064}


,alpha,eta,max_depth,min_child_weight,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
5,0.470520,0.269217,10.0,1.164722,xgboost-211111-0708-004-fed4f418,Completed,0.774009,2021-11-11 07:15:47+00:00,2021-11-11 07:16:47+00:00,60.0
3,1.842224,0.148015,8.0,3.010051,xgboost-211111-0708-006-a63e4233,Completed,0.758441,2021-11-11 07:16:05+00:00,2021-11-11 07:16:54+00:00,49.0
2,0.346252,0.066899,8.0,2.394268,xgboost-211111-0708-007-783c1ab0,Completed,0.755015,2021-11-11 07:20:25+00:00,2021-11-11 07:21:36+00:00,71.0
4,0.212607,0.265220,7.0,6.920360,xgboost-211111-0708-005-92562f25,Completed,0.743976,2021-11-11 07:15:56+00:00,2021-11-11 07:17:08+00:00,72.0
1,0.043518,0.062017,9.0,4.863994,xgboost-211111-0708-008-6c5eefd0,Completed,0.740969,2021-11-11 07:20:21+00:00,2021-11-11 07:21:28+00:00,67.0
0,0.928040,0.230840,10.0,1.637310,xgboost-211111-0708-009-2192602e,Completed,0.736896,2021-11-11 07:20:23+00:00,2021-11-11 07:21:13+00:00,50.0
7,1.569030,0.349067,7.0,4.695803,xgboost-211111-0708-002-a2a82061,Completed,0.723916,2021-11-11 07:11:10+00:00,2021-11-11 07:12:22+00:00,72.0
6,1.894026,0.791378,10.0,2.888466,xgboost-211111-0708-003-e9e1427f,Completed,0.723878,2021-11-11 07:11:27+00:00,2021-11-11 07:12:48+00:00,81.0
8,1.084758,0.684199,5.0,5.436294,xgboost-211111-0708-001-cb57f046,Completed,0.687526,2021-11-11 07:11:17+00:00,2021-11-11 07:12:13+00:00,56.0


## See TuningJob results vs time
Next we will show how the objective metric changes over time, as the tuning job progresses.  For Bayesian strategy, you should expect to see a general trend towards better results, but this progress will not be steady as the algorithm needs to balance _exploration_ of new areas of parameter space against _exploitation_ of known good areas.  This can give you a sense of whether or not the number of training jobs is sufficient for the complexity of your search space.

In [6]:
import bokeh
import bokeh.io

bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool


class HoverHelper:
    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append((k, "@{%s}" % k))

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools="pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset"):
        return [self.hovertool(), standard_tools]


hover = HoverHelper(tuner)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type="datetime")
p.circle(source=df, x="TrainingStartTime", y="FinalObjectiveValue")
show(p)

Loading BokehJS ...

## Analyze the correlation between objective metric and individual hyperparameters 
Now you have finished a tuning job, you may want to know the correlation between your objective metric and individual hyperparameters you've selected to tune. Having that insight will help you decide whether it makes sense to adjust search ranges for certain hyperparameters and start another tuning job. For example, if you see a positive trend between objective metric and a numerical hyperparameter, you probably want to set a higher tuning range for that hyperparameter in your next tuning job.

The following cell draws a graph for each hyperparameter to show its correlation with your objective metric.

In [7]:
ranges = tuner.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get("Values"):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0

        vals = hp_range["Values"]
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args["x_range"] = vals

    # Now plot it
    p = figure(
        plot_width=500,
        plot_height=500,
        title="Objective vs %s" % hp_name,
        tools=hover.tools(),
        x_axis_label=hp_name,
        y_axis_label=objective_name,
        **categorical_args,
    )
    p.circle(source=df, x=hp_name, y="FinalObjectiveValue")
    figures.append(p)
show(bokeh.layouts.Column(*figures))